<a href="https://colab.research.google.com/github/ebamberg/TechPackersColab/blob/main/examples_LLM_fine_tuning_using_SFT_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

see: https://huggingface.co/docs/trl/main/en/sft_trainer

In [1]:
!pip install transformers --upgrade --quiet
!pip install datasets --quiet
!pip install torch --quiet
# Attention: we have to install peft before trl
!pip install peft
!pip install -U bitsandbytes
!pip install accelerate
!pip install trl --quiet


In [2]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from transformers import AutoModelForCausalLM


In [7]:
dataset = load_dataset("imdb", split="train")

print (type(dataset))
print (dataset.features)
print (dataset.column_names)
print (dataset["text"][:3])

dataset = dataset[:10]


<class 'datasets.arrow_dataset.Dataset'>
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
['text', 'label']
['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What

In [ ]:
llmmodels =["facebook/opt-350m","EleutherAI/gpt-neo-125m"]

In [8]:
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
#    model_init_kwargs={
#        "torch_dtype": "bfloat16",
#    },
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-350m",
    load_in_4bit=True,
    device_map="auto",
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=sft_config,
    peft_config=peft_config
)
trainstats=trainer.train()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


AttributeError: 'dict' object has no attribute 'column_names'

In [ ]:
print (trainstats)

Example prediction

In [ ]:
#from transformers import pipeline

#generator = pipeline('text-generation', model="facebook/opt-350m")
#generator("What are we having for dinner?")